<a href="https://colab.research.google.com/github/blueCormorant/MCTest-Augmented-/blob/master/HF_Initial_Evals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from transformers import AutoTokenizer, AutoModel

In [2]:
from transformers import GPT2Tokenizer, AutoModelForCausalLM
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
dir_path = "drive/MyDrive/Thesis/"
dataset_name = "1200_prompts_dataset.tsv"

In [5]:
def load_model(model_name):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token_id = tokenizer.eos_token_id
    model = AutoModelForCausalLM.from_pretrained("gpt2")
    return tokenizer, model

In [21]:
def predict_next_token(tokenizer, model, prompt):
    inputs = tokenizer([prompt], return_tensors="pt")
    input_length = inputs.input_ids.shape[1]

    outputs = model.generate(
        **inputs,
        max_new_tokens=MAX_NEW_TOKENS,
        return_dict_in_generate=True,
        output_scores=True,
        num_return_sequences=NUM_RETURN_SEQUENCES,
        do_sample=False,
        prefix_allowed_tokens_fn=lambda batch_id, sent: [tokenizer.encode(token) for token in MCQA_options]
    )

    transition_scores = model.compute_transition_scores(
        outputs.sequences, outputs.scores, normalize_logits=True
    )

    return outputs.sequences[:, input_length:], transition_scores


In [7]:
def print_predictions(generated_tokens, transition_scores):
    print(f"| token | token string | logits | probability |")
    for i in range(len(list(zip(generated_tokens, transition_scores)))):
        for tok, score in zip(generated_tokens[i], transition_scores[i]):
            print(f"| {tok:5d} | {tokenizer.decode(tok):12s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}      |")

In [22]:
tokenizer, model = load_model("gpt2")

In [9]:
with open(f"{dir_path}{dataset_name}", "r") as infile:
    dataset = pd.read_csv(infile, sep="\t").drop(columns = ["Unnamed: 0"])

In [23]:
cluster0 = dataset[dataset["item"] == 0]

In [24]:
cluster0

,item,complexity_score,char_len,score_over_len,num_tokens,score_over_tokens,prompt,answers
0,0,52,925,0.056216,173,0.300578,Alyssa got to the beach after a long trip. She...,A : trip\nB : Miami\nC : Atlanta\nD : beach\n
1,0,54,936,0.057692,174,0.310345,Alyssa got to the beach after a long trip. She...,A : 1\nB : 2\nC : 3\nD : 4\n
2,0,53,929,0.057051,174,0.304598,Alyssa got to the beach after a long trip. She...,A : swim\nB : travel\nC : visit friends\nD : l...
3,0,52,940,0.055319,175,0.297143,Alyssa got to the beach after a long trip. She...,A : steak\nB : soup\nC : salad\nD : catfish\n


In [31]:
max(dataset["item"])

299

In [16]:
MCQA_options  = ["A", "B", "C", "D"]
NUM_RETURN_SEQUENCES = 1
MAX_NEW_TOKENS = 1

In [25]:
print(cluster0.iloc[0]["prompt"] + cluster0.iloc[0]["answers"])

Alyssa got to the beach after a long trip. She's from Charlotte. She traveled from Atlanta. She's now in Miami. She went to Miami to visit some friends. But she wanted some time to herself at the beach, so she went there first. After going swimming and laying out, she went to her friend Ellen's house. Ellen greeted Alyssa and they both had some lemonade to drink. Alyssa called her friends Kristin and Rachel to meet at Ellen's house. The girls traded stories and caught up on their lives. It was a happy time for everyone. The girls went to a restaurant for dinner. The restaurant had a special on catfish. Alyssa enjoyed the restaurant's special. Ellen ordered a salad. Kristin had soup. Rachel had a steak. After eating, the ladies went back to Ellen's house to have fun. They had lots of fun. They stayed the night because they were tired. Alyssa was happy to spend time with her friends again.
What city is Alyssa in?
A : trip
B : Miami
C : Atlanta
D : beach



In [91]:
def eval_prompt_cluster(tokenizer, model, cluster):
    pred = []
    for question_id in range(len(cluster)):
        prompt = cluster.iloc[question_id]["prompt"] + cluster.iloc[question_id]["answers"] + "Answer : "
        predictions, transition_scores = predict_next_token(tokenizer, model, prompt)
        pred_decoded = tokenizer.decode(predictions[0])
        prob_decoded = np.exp(transition_scores[0].numpy())[0]
        pred.append(pd.Series([pred_decoded, prob_decoded]))
    return pred

In [131]:
def eval_dataset(tokenizer, model, dataset):
    total_preds = []
    for cluster_id in range(max(dataset["item"])+1)[:3]:
        cluster = dataset[dataset["item"] == cluster_id]
        cluster_preds = eval_prompt_cluster(tokenizer, model, cluster)
        cluster_df = pd.DataFrame(cluster_preds).set_axis(["pred", "prob"], axis="columns")
        cluster_df["item"] = cluster_id
        total_preds.append(cluster_df)
        sorted_preds = pd.concat(total_preds).sort_values(by='item')
        reindexed_preds = sorted_preds.set_index(pd.Index(range(len(sorted_preds))))
        reindexed_preds = reindexed_preds[["item", "pred", "prob"]]
    return reindexed_preds

In [132]:
predictions = eval_dataset(tokenizer, model, dataset)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [133]:
predictions

,item,pred,prob
0,0,A,0.660971
1,0,A,0.725945
2,0,A,0.790948
3,0,A,0.648342
4,1,A,0.541876
5,1,A,0.594729
6,1,A,0.534933
7,1,A,0.578181
8,2,A,0.480415
9,2,A,0.474993
